In [ ]:
from collections import defaultdict, Counter
import json
import re
import numpy as np
import sys
import pickle
import random

np.set_printoptions(precision=2)

users = re.compile('@[^ ]+')
numbers = re.compile('[0-9]')
urls = re.compile("(https?:\/\/)?(?:www\.|(?!www))?[^\s\.]+\.[^\s]{2,}|(www)?\.[^\s]+\.[^\s]{2,}")

In [ ]:
class StructuredPerceptron(object):
    """
    implements a structured perceptron as described in Collins 2002,
    with updates from https://explosion.ai/blog/part-of-speech-pos-tagger-in-python
    """

    def __init__(self):
        """
        initialize model parameters
        """
        self.tags = set()
        self.feature_weights = defaultdict(lambda: defaultdict(float)) #feature_name -> tags -> weight
        self.weight_totals = defaultdict(lambda: defaultdict(float)) #feature_name -> tags -> weight
        self.timestamps = defaultdict(lambda: defaultdict(float)) #feature_name -> tags -> weight

        self.tag_dict = defaultdict(set) #word -> {tags}

        self.START = "__START__"
        self.END = "__END__"

        self.Q = None
        self.backpointers = None

        # self.instances = None
        
        
    def normalize(self, word):
        """
        lowercase word, and replace numbers, user names, and URLs
        """
        return re.sub(urls, 'URL', re.sub(users, '@USER', re.sub(numbers, '0', word.strip().lower())))

    
    def evaluate(self, data_instances, method='greedy'):
        correct = 0
        total = 0
        for (words, tags) in data_instances:
            preds = self.predict(words, method=method)
            matches = sum(map(lambda x: int(x[0]==x[1]), zip(preds, tags)))
            correct += matches
            total += len(tags)
        return correct/total
        
    
    def fit(self, file_name, dev_file=None, iterations=10, learning_rate=0.25, inference='greedy', verbose=False):
        """
        read in a CoNLL-format file, extract features to train weight vector
        """        
        # initialize tag dictionary for each word and get tag set
        self.instances = [(words, tags) for (words, tags) in self.read_conll_file(file_name)]
        instances = self.instances
        for (words, tags) in instances:
            self.tags.update(set(tags))

            for word, tag in zip(words, tags):
                self.tag_dict[self.normalize(word)].add(tag)
        
        if dev_file:
            dev_instances = [(words, tags) for (words, tags) in self.read_conll_file(dev_file)]
            
        # iterate over data
        for iteration in range(1, iterations+1):
            correct = 0
            total = 0
            if verbose:
                print('Iteration {}'.format(iteration+1), file=sys.stderr, flush=True)
                print("*" * 15, file=sys.stderr, flush=True)

            random.shuffle(instances)
            for i, (words, tags) in enumerate(instances):
                if i > 0:
                    if i%1000==0:
                        print('%s'%i, file=sys.stderr, flush=True)
                    elif i%20==0:
                        print('.', file=sys.stderr, flush=True, end='')

                # get prediction
                prediction = self.predict(words, method=inference)

                # derive global features
                global_gold_features, global_prediction_features = self.get_global_features(words, prediction, tags)
                                    
                # update weight vector:
                # 1. move closer to true tag
                for tag, fids in global_gold_features.items():
                    for fid, count in fids.items():
                        nr_iters_at_this_weight = iteration - self.timestamps[fid][tag]
                        self.weight_totals[fid][tag] += nr_iters_at_this_weight * self.feature_weights[fid][tag]
                        self.timestamps[fid][tag] = iteration
                        self.feature_weights[fid][tag] += learning_rate * count

                # 2. move further from wrong tag
                for tag, fids in global_prediction_features.items():
                    for fid, count in fids.items():
                        nr_iters_at_this_weight = iteration - self.timestamps[fid][tag]
                        self.weight_totals[fid][tag] += nr_iters_at_this_weight * self.feature_weights[fid][tag]
                        self.timestamps[fid][tag] = iteration
                        self.feature_weights[fid][tag] -= learning_rate * count
                        
                # compute training accuracy for this iteration
                correct += sum([int(predicted_tag == true_tag) for predicted_tag, true_tag in zip(prediction, tags)])
                total += len(tags)

                # output examples
                if verbose and i%1000==0:
                    print("current word accuracy:{:.2f}".format(correct/total))
                    print(list(zip(words, 
                                   [self.normalize(word) for word in words], 
                                   tags, 
                                   prediction)), file=sys.stderr, flush=True)
            
            print('\t{} features'.format(len(self.feature_weights)), file=sys.stderr, flush=True)
            print('\tTraining accuracy: {:.2f}\n'.format(correct/total), file=sys.stderr, flush=True)
            if dev_file:
                print('\tDevelopment accuracy: {:.2f}\n'.format(self.evaluate(dev_instances, method=inference)), file=sys.stderr, flush=True)
         
        # average weights
        for feature, tags in self.feature_weights.items():
            for tag in tags:
                total = self.weight_totals[feature][tag]
                total += (iterations - self.timestamps[feature][tag]) * self.feature_weights[feature][tag]
                averaged = round(total / float(iterations), 3)
                self.feature_weights[feature][tag] = averaged


    def get_features(self, word, previous_tag2, previous_tag, words, i):
        """
        get all features that can be derived from the word and previous tags
        """
        prefix = word[:3]
        suffix = word[-3:]

        features = {
                    'PREFIX={}'.format(prefix),
                    'SUFFIX={}'.format(suffix),
                    'LEN<=3={}'.format(len(word)<=3),
                    'FIRST_LETTER={}'.format(word[0]),
                    'WORD={}'.format(word),
                    'NORM_WORD={}'.format(words[i]),
                    'PREV_WORD={}'.format(words[i-1]),
                    'PREV_WORD_PREFIX={}'.format(words[i-1][:3]),
                    'PREV_WORD_SUFFIX={}'.format(words[i-1][-3:]),
                    'PREV_WORD+WORD={}+{}'.format(words[i-1], words[i]),
                    'NEXT_WORD={}'.format(words[i+1]),
                    'NEXT_WORD_PREFIX={}'.format(words[i+1][:3]),
                    'NEXT_WORD_SUFFIX={}'.format(words[i+1][-3:]),
                    'WORD+NEXT_WORD={}'.format(word, words[i+1]),
                    'NEXT_2WORDS={}+{}'.format(words[i+1], words[i+2]),
                    'PREV_TAG={}'.format(previous_tag),                 # previous tag
                    'PREV_TAG2={}'.format(previous_tag2),                 # two-previous tag
                    'PREV_TAG_BIGRAM={}+{}'.format(previous_tag2, previous_tag),  # tag bigram
                    'PREV_TAG+WORD={}+{}'.format(previous_tag, word),            # word-tag combination
                    'PREV_TAG+PREFIX={}_{}'.format(previous_tag, prefix),        # prefix and tag
                    'PREV_TAG+SUFFIX={}_{}'.format(previous_tag, suffix),        # suffix and tag
                    'WORD+TAG_BIGRAM={}+{}+{}'.format(word, previous_tag2, previous_tag),
                    'SUFFIX+2TAGS={}+{}+{}'.format(suffix, previous_tag2, previous_tag),
                    'PREFIX+2TAGS={}+{}+{}'.format(prefix, previous_tag2, previous_tag),
                    'BIAS'
            }
        return features
    
    
    def get_global_features(self, words, predicted_tags, true_tags):
        '''
        sum up local features
        '''
        context = [self.START] + [self.normalize(word) for word in words] + [self.END, self.END]

        global_gold_features = defaultdict(lambda: Counter())
        global_prediction_features = defaultdict(lambda: Counter())

        prev_predicted_tag = self.START
        prev_predicted_tag2 = self.START
        
        for j, (word, predicted_tag, true_tag) in enumerate(zip(words, predicted_tags, true_tags)):
            # get the predicted features. NB: use j+1, since context is longer than words
            prediction_features = self.get_features(word, prev_predicted_tag2, prev_predicted_tag, context, j+1)

            # update feature correlation with true and predicted tag
            global_prediction_features[predicted_tag].update(prediction_features)
            global_gold_features[true_tag].update(prediction_features)

            prev_predicted_tag2 = prev_predicted_tag
            prev_predicted_tag = predicted_tag

        return global_gold_features, global_prediction_features
            
    
    def get_scores(self, features):
        """
        predict scores for each tag given features
        """
        scores = defaultdict(float)
        
        # add up the scores for each tag
        for feature in features:
            if feature not in self.feature_weights:
                continue
            weights = self.feature_weights[feature]
            for tag, weight in weights.items():
                scores[tag] += weight

        # return tag scores
        if not scores:
            # if there are no scores (e.g., first iteration),
            # simply return the first tag with score 1
            scores[list(self.tags)[0]] = 1
        
        return scores


    def predict(self, words, method='greedy'):
        '''
        predict tags using one of two methods
        '''
        if method == 'greedy':
            return self.predict_greedy(words)
        elif method == 'viterbi':
            return self.predict_viterbi(words)


    def predict_viterbi(self, words):
        '''
        predict using Viterbi decoding
        '''
        context = [self.START] + [self.normalize(word) for word in words] + [self.END, self.END]

        N = len(words)
        M = len(self.tags) #number of tags
        tags = sorted(self.tags)

        # create trellis of size M (number of tags) x N (sentence length)
        self.Q = np.ones((M, N)) * float('-Inf')
        self.backpointers = np.ones((M, N), dtype=np.int16) * -1 #backpointers

        # initialize probs for tags j at position 1 (first word)
        features = self.get_features(words[0], self.START, self.START, context, 1)
        scores = self.get_scores(features)
        allowed_initial_tags = self.tag_dict[context[1]]

        for j in range(M):
            if not allowed_initial_tags or tags[j] in allowed_initial_tags:
                Q[j,0] = scores[tags[j]]

        # filling the lattice, for every position and every tag find viterbi score Q
        for i in range(1, N):
            allowed_tags = self.tag_dict[context[i+1]]

            # for every previous tag
            for j in range(M):
                best_score = 0.0 #float('-Inf')
                prev_tag = tags[j]

                # skip impossible tags
                allowed_previous_tags = self.tag_dict[context[i]]
                if allowed_previous_tags and prev_tag not in allowed_previous_tags:
                    continue

                best_before = self.Q[j,i-1] # score of previous tag

                # for every possible pre-previous tag
                for k in range(M):
                    if i == 1:
                        prev2_tag = self.START
                    else:
                        prev2_tag = tags[k]
                        # skip impossible tags
                        allowed_previous2_tags = self.tag_dict[context[i-1]]
                        if allowed_previous2_tags and prev2_tag not in allowed_previous2_tags:
                            continue

                    # get features of word i with the two previous tags
                    features = self.get_features(words[i], prev2_tag, prev_tag, context, i+1)
                    scores = self.get_scores(features)

                    # update best score
                    for t in range(M):
                        tag = tags[t]
                        # if word is unknown, use all tags, otherwise allowed ones
                        if not allowed_tags or tag in allowed_tags:
                            tag_score = best_before + scores[tag]

                            if tag_score > best_score:
                                self.Q[t,i] = tag_score
                                best_score = tag_score
                                self.backpointers[t,i] = j

        # final best
        best_id = self.Q[:,-1].argmax()

        # print best tags in reverse order
        predtags = [tags[best_id]]

        for i in range(N-1,0,-1):
            idx = self.backpointers[best_id, i]
            predtags.append(tags[idx])
            best_id = idx

        #return reversed predtags
        return predtags[::-1]         

    
    def predict_greedy(self, words):
        '''
        greedy prediction
        '''
        context = [self.START] + [self.normalize(word) for word in words] + [self.END, self.END]
                
        prev_predicted_tag = self.START
        prev_predicted_tag2 = self.START

        out = []

        for j, word in enumerate(words):
            # for unambiguous words, just look up the tag
            predicted_tag = list(self.tag_dict[context[j+1]])[0] if len(self.tag_dict[context[j+1]]) == 1 else None

            if not predicted_tag:
                # get the predicted features. NB: use j+1, since context is longer than words
                prediction_features = self.get_features(word, prev_predicted_tag2, prev_predicted_tag, context, j+1)
                scores = self.get_scores(prediction_features)
                
                # predict the current tag
                predicted_tag = max(scores, key=scores.get)

            prev_predicted_tag2 = prev_predicted_tag
            prev_predicted_tag = predicted_tag

            out.append(predicted_tag)

        return out
    
        
    def read_conll_file(self, file_name):
        """
        read in a file with CoNLL format:
        word1    tag1
        word2    tag2
        ...      ...
        wordN    tagN

        Sentences MUST be separated by newlines!
        """
        current_words = []
        current_tags = []

        with open(file_name, encoding='utf-8') as conll:
            for line in conll:
                line = line.strip()

                if line:
                    word, tag = line.split('\t')
                    current_words.append(word)
                    current_tags.append(tag)

                else:
                    yield (current_words, current_tags)
                    current_words = []
                    current_tags = []

        # if file does not end in newline (it should...), check whether there is an instance in the buffer
        if current_tags != []:
            yield (current_words, current_tags)
        

    def save(self, file_name):
        """
        save model as pickle file
        """
        print("saving model...", end=' ', file=sys.stderr)
        with open(file_name, "wb") as model:
            # pickle cannot save default_dictionaries
            # => make copy and turn into regular dictionaries
            save_feature_weights = defaultdict(lambda: defaultdict(float))
            save_feature_weights.update(self.feature_weights)
            save_tag_dict = defaultdict(set)
            save_tag_dict.update(self.tag_dict)

            save_feature_weights.default_factory = None
            save_tag_dict.default_factory = None
            pickle.dump((save_feature_weights, save_tag_dict, self.tags),
                     model, -1)
        print("done", file=sys.stderr)


    def load(self, file_name):
        """
        load model from pickle file
        """
        print("loading model...", end=' ', file=sys.stderr)
        with open(file_name, 'rb') as model:
            try:
                parameters = pickle.load(model)
            except IOError:
                msg = ("No such model file.")
                raise MissingCorpusError(msg)

            feature_weights, tag_dict, tags = parameters
            self.tags = tags

            # pickle cannot store defaultdicts, so we need a 2-step process
            # 1. initialize
            self.feature_weights = defaultdict(lambda: defaultdict(float))
            self.tag_dict = defaultdict(set)
            
            # 2. update
            self.feature_weights.update(feature_weights)
            self.tag_dict.update(tag_dict)
        print("done", file=sys.stderr)
        return None

In [ ]:
sp = StructuredPerceptron()
inference_method = 'greedy'
%time sp.fit('tiny_POS_train.data', dev_file='tiny_POS_test.data', iterations=10, inference=inference_method)
sp.save('model_greedy.pickle')


.................................................1000
.................................................2000
	181847 features
	Training accuracy: 0.95

	Development accuracy: 0.91

.................................................1000
.................................................2000
	187490 features
	Training accuracy: 0.98

	Development accuracy: 0.91

.................................................1000
.................................................2000
	189026 features
	Training accuracy: 0.99

	Development accuracy: 0.92

.................................................1000
.................................................2000
	189671 features
	Training accuracy: 0.99

	Development accuracy: 0.92

.................................................1000
.................................................2000
	190056 features
	Training accuracy: 0.99

	Development accuracy: 0.92

.................................................1000
..............................................

CPU times: user 1min 33s, sys: 757 ms, total: 1min 34s
Wall time: 1min 35s


saving model... done


## DEBUG GREEDY

In [ ]:
sp.tags

{'.',
 'ADJ',
 'ADP',
 'ADV',
 'CONJ',
 'DET',
 'NOUN',
 'NUM',
 'PRON',
 'PRT',
 'VERB',
 'X'}

In [ ]:
sp.tag_dict

defaultdict(set,
            {'showtime': {'NOUN'},
             'is': {'VERB'},
             'a': {'DET', 'NOUN'},
             'distant': {'ADJ'},
             'no.': {'NOUN'},
             '0': {'NUM'},
             'to': {'ADP', 'PRT'},
             'home': {'NOUN'},
             'box': {'NOUN'},
             'office': {'NOUN'},
             ',': {'.'},
             'and': {'CONJ'},
             'in': {'ADP', 'ADV', 'PRT'},
             'may': {'NOUN', 'VERB'},
             'filed': {'VERB'},
             '$': {'.'},
             '0.0': {'NUM'},
             'billion': {'NUM'},
             'antitrust': {'ADJ'},
             'suit': {'ADV', 'NOUN'},
             'against': {'ADP'},
             'time': {'NOUN'},
             'warner': {'NOUN'},
             'charging': {'VERB'},
             'the': {'DET', 'NOUN'},
             'company': {'NOUN'},
             'its': {'PRON'},
             'hbo': {'NOUN'},
             'american': {'ADJ', 'NOUN'},
             'television': {'NOUN

In [ ]:
print('TEXT:',' '.join(sp.instances[3][0]))
print('TAGS:',' '.join(sp.instances[3][1]))

(list(zip(sp.instances[3][0],sp.instances[3][1])))

TEXT: He said Hydro-Quebec already has some `` customers in mind '' for the power that was to be delivered to Maine .
TAGS: PRON VERB NOUN ADV VERB DET . NOUN ADP NOUN . ADP DET NOUN DET VERB PRT VERB VERB ADP NOUN .


[('He', 'PRON'),
 ('said', 'VERB'),
 ('Hydro-Quebec', 'NOUN'),
 ('already', 'ADV'),
 ('has', 'VERB'),
 ('some', 'DET'),
 ('``', '.'),
 ('customers', 'NOUN'),
 ('in', 'ADP'),
 ('mind', 'NOUN'),
 ("''", '.'),
 ('for', 'ADP'),
 ('the', 'DET'),
 ('power', 'NOUN'),
 ('that', 'DET'),
 ('was', 'VERB'),
 ('to', 'PRT'),
 ('be', 'VERB'),
 ('delivered', 'VERB'),
 ('to', 'ADP'),
 ('Maine', 'NOUN'),
 ('.', '.')]

In [ ]:
sp.feature_weights

defaultdict(<function __main__.StructuredPerceptron.__init__.<locals>.<lambda>>,
            {'NEXT_WORD=marshall': defaultdict(float,
                         {'.': 0.0,
                          'ADP': -0.025,
                          'CONJ': 0.0,
                          'DET': 0.0,
                          'NOUN': 0.0,
                          'PRT': 0.025}),
             'PREV_WORD+WORD=__START__+george': defaultdict(float,
                         {'NOUN': 0.0}),
             'BIAS': defaultdict(float,
                         {'.': -0.1,
                          'ADJ': 0.6,
                          'ADP': -0.65,
                          'ADV': 0.025,
                          'CONJ': 0.0,
                          'DET': -0.325,
                          'NOUN': 1.0,
                          'NUM': -0.25,
                          'PRON': -0.025,
                          'PRT': -0.75,
                          'VERB': 0.475,
                          'X': 0.0}),
       

In [ ]:
words = sp.instances[3][0]
tags = sp.instances[3][1]
print("WORDS",words)
print("TAGS",tags)

context = [sp.START] + [sp.normalize(word) for word in words] + [sp.END, sp.END]

print("CONTEXT",context)

prediction_features = sp.get_features("said", "__START__", "PRON", context , 2)

print("FEATURES",prediction_features)

scores = sp.get_scores(prediction_features)

print("SCORES",scores)

# predict the current tag
predicted_tag = max(scores, key=scores.get)

print("PREDICTED TAG",predicted_tag)

WORDS ['He', 'said', 'Hydro-Quebec', 'already', 'has', 'some', '``', 'customers', 'in', 'mind', "''", 'for', 'the', 'power', 'that', 'was', 'to', 'be', 'delivered', 'to', 'Maine', '.']
TAGS ['PRON', 'VERB', 'NOUN', 'ADV', 'VERB', 'DET', '.', 'NOUN', 'ADP', 'NOUN', '.', 'ADP', 'DET', 'NOUN', 'DET', 'VERB', 'PRT', 'VERB', 'VERB', 'ADP', 'NOUN', '.']
CONTEXT ['__START__', 'he', 'said', 'hydro-quebec', 'already', 'has', 'some', '``', 'customers', 'in', 'mind', "''", 'for', 'the', 'power', 'that', 'was', 'to', 'be', 'delivered', 'to', 'maine', '.', '__END__', '__END__']
FEATURES {'PREFIX+2TAGS=sai+__START__+PRON', 'WORD+TAG_BIGRAM=said+__START__+PRON', 'PREV_WORD+WORD=he+said', 'PREV_TAG+PREFIX=PRON_sai', 'BIAS', 'PREV_WORD_PREFIX=he', 'NEXT_WORD_PREFIX=hyd', 'SUFFIX=aid', 'PREV_WORD_SUFFIX=he', 'PREV_TAG_BIGRAM=__START__+PRON', 'WORD+NEXT_WORD=said', 'WORD=said', 'NEXT_WORD_SUFFIX=bec', 'LEN<=3=False', 'SUFFIX+2TAGS=aid+__START__+PRON', 'PREV_TAG+WORD=PRON+said', 'PREV_TAG+SUFFIX=PRON_aid'

In [ ]:
prediction_features

{'BIAS',
 'FIRST_LETTER=s',
 'LEN<=3=False',
 'NEXT_2WORDS=hydro-quebec+already',
 'NEXT_WORD=hydro-quebec',
 'NEXT_WORD_PREFIX=hyd',
 'NEXT_WORD_SUFFIX=bec',
 'NORM_WORD=said',
 'PREFIX+2TAGS=sai+__START__+PRON',
 'PREFIX=sai',
 'PREV_TAG+PREFIX=PRON_sai',
 'PREV_TAG+SUFFIX=PRON_aid',
 'PREV_TAG+WORD=PRON+said',
 'PREV_TAG2=__START__',
 'PREV_TAG=PRON',
 'PREV_TAG_BIGRAM=__START__+PRON',
 'PREV_WORD+WORD=he+said',
 'PREV_WORD=he',
 'PREV_WORD_PREFIX=he',
 'PREV_WORD_SUFFIX=he',
 'SUFFIX+2TAGS=aid+__START__+PRON',
 'SUFFIX=aid',
 'WORD+NEXT_WORD=said',
 'WORD+TAG_BIGRAM=said+__START__+PRON',
 'WORD=said'}

In [ ]:
prediction_features

In [ ]:
prediction = sp.predict(words, method="greedy")
global_gold_features, global_prediction_features = sp.get_global_features(words, prediction, tags)

print(global_gold_features['DET'])
print(global_prediction_features['DET'])

Counter({'BIAS': 3, 'LEN<=3=False': 2, 'FIRST_LETTER=t': 2, 'PREV_TAG+WORD=VERB+some': 1, 'WORD+TAG_BIGRAM=some+ADV+VERB': 1, 'PREV_TAG=VERB': 1, 'PREV_WORD=has': 1, 'PREV_TAG+PREFIX=VERB_som': 1, 'PREV_WORD_SUFFIX=has': 1, 'WORD+NEXT_WORD=some': 1, 'NEXT_2WORDS=``+customers': 1, 'NORM_WORD=some': 1, 'SUFFIX+2TAGS=ome+ADV+VERB': 1, 'PREFIX=som': 1, 'FIRST_LETTER=s': 1, 'NEXT_WORD=``': 1, 'NEXT_WORD_SUFFIX=``': 1, 'PREV_WORD_PREFIX=has': 1, 'PREV_TAG+SUFFIX=VERB_ome': 1, 'PREV_TAG2=ADV': 1, 'NEXT_WORD_PREFIX=``': 1, 'PREFIX+2TAGS=som+ADV+VERB': 1, 'SUFFIX=ome': 1, 'PREV_TAG_BIGRAM=ADV+VERB': 1, 'WORD=some': 1, 'PREV_WORD+WORD=has+some': 1, 'PREV_WORD+WORD=for+the': 1, 'PREV_TAG+SUFFIX=ADP_the': 1, 'SUFFIX=the': 1, 'PREFIX=the': 1, 'PREV_WORD=for': 1, 'NEXT_WORD_SUFFIX=wer': 1, 'PREV_TAG=ADP': 1, 'LEN<=3=True': 1, 'NEXT_2WORDS=power+that': 1, 'PREV_TAG2=.': 1, 'NORM_WORD=the': 1, 'WORD+NEXT_WORD=the': 1, 'WORD=the': 1, 'PREV_TAG_BIGRAM=.+ADP': 1, 'PREV_WORD_PREFIX=for': 1, 'PREFIX+2TAGS=

In [ ]:
iteration = 1
for tag, fids in global_gold_features.items():
  print(tag, fids)
  for fid, count in fids.items():
    nr_iters_at_this_weight = iteration - sp.timestamps[fid][tag]
        # self.weight_totals[fid][tag] += nr_iters_at_this_weight * self.feature_weights[fid][tag]
        # self.timestamps[fid][tag] = iteration
        # self.feature_weights[fid][tag] += learning_rate * count

PRON Counter({'NORM_WORD=he': 1, 'PREV_TAG+PREFIX=__START___He': 1, 'SUFFIX=He': 1, 'PREV_TAG+WORD=__START__+He': 1, 'FIRST_LETTER=H': 1, 'NEXT_2WORDS=said+hydro-quebec': 1, 'SUFFIX+2TAGS=He+__START__+__START__': 1, 'BIAS': 1, 'PREV_WORD=__START__': 1, 'LEN<=3=True': 1, 'PREV_TAG_BIGRAM=__START__+__START__': 1, 'PREFIX=He': 1, 'PREV_TAG+SUFFIX=__START___He': 1, 'PREV_WORD_PREFIX=__S': 1, 'PREV_WORD+WORD=__START__+he': 1, 'WORD+TAG_BIGRAM=He+__START__+__START__': 1, 'WORD=He': 1, 'PREV_TAG=__START__': 1, 'PREV_WORD_SUFFIX=T__': 1, 'NEXT_WORD=said': 1, 'NEXT_WORD_SUFFIX=aid': 1, 'PREV_TAG2=__START__': 1, 'NEXT_WORD_PREFIX=sai': 1, 'PREFIX+2TAGS=He+__START__+__START__': 1, 'WORD+NEXT_WORD=He': 1})
VERB Counter({'BIAS': 5, 'LEN<=3=True': 3, 'LEN<=3=False': 2, 'PREV_TAG2=NOUN': 2, 'NEXT_WORD_PREFIX=to': 2, 'NEXT_WORD_SUFFIX=to': 2, 'NEXT_WORD=to': 2, 'PREFIX+2TAGS=sai+__START__+PRON': 1, 'WORD+TAG_BIGRAM=said+__START__+PRON': 1, 'PREV_WORD+WORD=he+said': 1, 'PREV_TAG+PREFIX=PRON_sai': 1, 'P

In [ ]:
list(zip(prediction,tags))

[('PRON', 'PRON'),
 ('VERB', 'VERB'),
 ('NOUN', 'NOUN'),
 ('ADV', 'ADV'),
 ('VERB', 'VERB'),
 ('DET', 'DET'),
 ('.', '.'),
 ('NOUN', 'NOUN'),
 ('ADP', 'ADP'),
 ('NOUN', 'NOUN'),
 ('.', '.'),
 ('ADP', 'ADP'),
 ('DET', 'DET'),
 ('NOUN', 'NOUN'),
 ('DET', 'DET'),
 ('VERB', 'VERB'),
 ('PRT', 'PRT'),
 ('VERB', 'VERB'),
 ('VERB', 'VERB'),
 ('ADP', 'ADP'),
 ('NOUN', 'NOUN'),
 ('.', '.')]

# VITERBI

In [ ]:
sp2 = StructuredPerceptron()
inference_method = 'viterbi'
%time sp2.fit('tiny_POS_train.data', dev_file='tiny_POS_test.data', iterations=5, inference=inference_method)

sp2.save('model_viterbi.pickle')

.................................................1000
.................................................2000
	127176 features
	Training accuracy: 0.04

	Development accuracy: 0.04

.................................................1000
.................................................2000
	127176 features
	Training accuracy: 0.04

	Development accuracy: 0.04

.................................................1000
.................................................2000
	127176 features
	Training accuracy: 0.04

	Development accuracy: 0.04

.................................................1000
.................................................2000
	127176 features
	Training accuracy: 0.04

	Development accuracy: 0.04

.................................................1000
.................................................2000
	127176 features
	Training accuracy: 0.04

	Development accuracy: 0.04



CPU times: user 1min 14s, sys: 491 ms, total: 1min 14s
Wall time: 1min 15s


saving model... done


## Debug

In [ ]:
# sp2
allowed_initial_tags = sp2.tag_dict[context[1]]
print(context)
print(context[1])
allowed_initial_tags

['__START__', 'he', 'said', 'hydro-quebec', 'already', 'has', 'some', '``', 'customers', 'in', 'mind', "''", 'for', 'the', 'power', 'that', 'was', 'to', 'be', 'delivered', 'to', 'maine', '.', '__END__', '__END__']
he


{'PRON'}

In [ ]:
N = len(words)
M = len(sp2.tags) #number of tags
tags = sorted(sp2.tags)

print('TAGS',tags)

# create trellis of size M (number of tags) x N (sentence length)
Q = np.ones((M, N)) * float('-Inf')

for j in range(M):
  if not allowed_initial_tags or tags[j] in allowed_initial_tags:
      Q[j,0] = scores[tags[j]]

Q

TAGS ['.', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM', 'PRON', 'PRT', 'VERB', 'X']


array([[ -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,
         -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,
         -inf,  -inf,  -inf,  -inf],
       [ -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,
         -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,
         -inf,  -inf,  -inf,  -inf],
       [ -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,
         -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,
         -inf,  -inf,  -inf,  -inf],
       [ -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,
         -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,
         -inf,  -inf,  -inf,  -inf],
       [ -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,
         -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,
         -inf,  -inf,  -inf,  -inf],
       [ -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,  -inf,
         -inf,  -inf,  -inf,  -inf

# Prediction

In [ ]:
# In order to load old models, use this:
# sp = StructuredPerceptron()
# sp.load('model_greedy.pickle')
# sp2 = StructuredPerceptron()
# sp2.load('model_viterbi.pickle')

print(sp.predict('Their management plan reforms worked'.split(), method='greedy'))
print(sp2.predict('Their management plan reforms worked'.split(), method='viterbi'))

print(sp.predict("Attack was their best option".split(), method='greedy'))
print(sp2.predict("Attack was their best option".split(), method='viterbi'))


print(sp.predict("This can can fly away".split(), method='greedy'))
print(sp.predict("This can can fly away".split(), method='viterbi'))

['PRON', 'NOUN', 'NOUN', 'VERB', 'VERB']
['X', 'X', 'X', 'X', '.']
['NOUN', 'VERB', 'PRON', 'ADJ', 'NOUN']
['X', 'X', 'X', 'X', '.']
['DET', 'VERB', 'VERB', 'VERB', 'ADV']
['X', 'X', 'X', 'X', '.']


## Exercise
Train one structured perceptron with greedy inference and one with Viterbi inference.
Compare training time and training accuracy.

Load the file `en-ud-dev.conll`. Tag it with both the Structured Perceptrons, compute the respective accuracy, and compare them.


In [ ]:
# your code here

## Exercise
Come up with more features, train a new model, and see whether you improved performance on the development set.


In [ ]:
# your code here